In [1]:
import sys
print(sys.version_info)

sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)


In [2]:
def stack(a, b):
    #pre = dict(hand = a, isTop = dict(a = False, b = True), on = dict(a = "hand"))
    #effect = dict(hand = "", isTop = dict(a = True, b = False), on = dict(a = b))
    pre = {
       'hand': a,
       'isTop':{a:False, b:True},
       'on':{a:"hand"}
    }
    effect = {
       'hand': "",
       'isTop':{a:True, b:False},
       'on':{a:b}
    }
    name = "stack" + a + b
    return makeAction(name, pre, effect)


def pickup(a):
    #pre = dict(hand = "", isTop = dict(a = True), on = dict(a = "table"))
    #effect = dict(hand = a, isTop = dict(a = False), on = dict(a = "hand"))
    pre = {
       'hand': "",
       'isTop':{a:True},
       'on':{a:"table"}
    }
   
    effect = {
       'hand': a,
       'isTop':{a:False},
       'on':{a:"hand"}
    }
    name = "pickup" + a
    return makeAction(name,pre, effect)

def putdown(a):
    #pre = dict(hand = a, isTop = dict(a = False), on = dict(a = "hand"))
    #effect = dict(hand = "", isTop = dict(a = True), on = dict(a = "table"))
    pre = {
       'hand': a,
       'isTop':{a:False},
       'on':{a:"hand"}
    }
    effect = {
       'hand': "",
       'isTop':{a:True},
       'on':{a:"table"}
    }
    name = "putdown" + a
    return makeAction(name, pre, effect)

def unstack(a, b):
    #pre = dict(hand = "", isTop = dict(a = True, b = False), on = dict(a = b))
    pre = {
        'hand':"",
        'isTop': {a:True, b:False},
        'on':{a:b}
          }
    effect = dict(hand = a, isTop= dict(a = False, b = True), on = dict(a = "hand"))
    effect = {
        'hand':a,
        'isTop':{a:False, b:True},
        'on':{a:"hand"}
    }
    name = "unstack" + a + b
    return makeAction(name, pre, effect)
    
def makeAction(name, precondition, effect, cost = 0):
    return dict(name = name, precondition = precondition, effect = effect, cost = cost)

In [3]:
class Node:
    def __init__(self, state, cost=0, act=None, nxt=0, score=0, diff=None, diffcount=0):
        self.state = state
        self.act = act
        self.nxt = nxt
        self.cost = cost
        self.score = score
        self.diff = diff
        self.diffcount = diffcount
    

In [4]:
domain = [stack("a", "b"),#0
          stack("a", "c"),
          stack("a", "d"),
          stack("b", "a"),#3
          stack("b", "c"),
          stack("b", "d"),
          stack("c", "a"),
          stack("c", "b"),
          stack("c", "d"),
          stack("d","a"),
          stack("d","b"),
          stack("d","c"),
          unstack("a", "b"),#12
          unstack("a", "c"),
          unstack("a", "d"),
          unstack("b", "a"),#15
          unstack("b", "c"),
          unstack("b", "d"),
          unstack("c", "a"),
          unstack("c", "b"),
          unstack("c", "d"),
          unstack("d","a"),
          unstack("d","b"),
          unstack("d","c"),
          pickup("a"),
          pickup("b"),
          pickup("c"),
          pickup("d"),
          putdown("a"),
          putdown("b"),
          putdown("c"),
          putdown("d")]
print(len(domain))
#print("list の番号　listの中身")
#for i, a in enumerate(domain):
#    print(i,a)
#print("effectのアクセス")
print(domain[0])
print(domain[3])
#print(domain[12])
#print(domain[15])
for key, var in domain[0].items():
    print("---key---")
    print(key)
    print("---var--")
    print(var)
    if key == 'effect':
        print("-------effect-------")
        for state_key, state_var in var.items():
            print("---state_key---")
            print(state_key)
            print("---state_var--")
            print(state_var)

            

32
{'name': 'stackab', 'precondition': {'hand': 'a', 'isTop': {'a': False, 'b': True}, 'on': {'a': 'hand'}}, 'effect': {'hand': '', 'isTop': {'a': True, 'b': False}, 'on': {'a': 'b'}}, 'cost': 0}
{'name': 'stackba', 'precondition': {'hand': 'b', 'isTop': {'b': False, 'a': True}, 'on': {'b': 'hand'}}, 'effect': {'hand': '', 'isTop': {'b': True, 'a': False}, 'on': {'b': 'a'}}, 'cost': 0}
---key---
name
---var--
stackab
---key---
precondition
---var--
{'hand': 'a', 'isTop': {'a': False, 'b': True}, 'on': {'a': 'hand'}}
---key---
effect
---var--
{'hand': '', 'isTop': {'a': True, 'b': False}, 'on': {'a': 'b'}}
-------effect-------
---state_key---
hand
---state_var--

---state_key---
isTop
---state_var--
{'a': True, 'b': False}
---state_key---
on
---state_var--
{'a': 'b'}
---key---
cost
---var--
0


In [5]:
def getDiffState(dest, src):
    diff = {}
    count = 0
    for key,var in dest.items():
        if isinstance(var, dict):
            tbl, cnt = getDiffState(var, src[key])
            if cnt > 0:
                diff[key] = tbl
                count = count + cnt
        elif src.get(key) != None:
            if src[key] != dest[key]:
                diff[key] = var
                count = count + 1
        elif src.get(key) == None:
            diff[key] = var
            count = count +1
    return diff, count

A = stack("b","a")
goal = dict(on = dict(b = "a", a = "table"))
print("[A]", A['effect'])
print("[goal]", goal)
diff, diffcount = getDiffState(goal, A['effect'])
print("diff",diff ) # a:table　が入るはず
print("diffcount", diffcount)

B = putdown("a")
goal = dict(on = dict(b = "a", a = "table"))
print("[B]", B['effect'])
print("[goal]", goal)
diff, diffcount = getDiffState(goal, B['effect'])
print("diff",diff ) # b:a　が入るはず
print("diffcount", diffcount)

goal = dict(hand="b", isTop=dict(a=True, b=False), on=dict(a="table", b="hand"))
C = dict(hand="b", on = dict(b = "hand"), isTop=dict(b=False))
print("[C]", C)
print("[goal]", goal)
diff, diffcount = getDiffState(goal, C)
print("diff",diff ) #  on a:Table, isTop a:True　が入るはず
print("diffcount", diffcount)

def isInState_internal(a, b):
    same_state_count = 0
    for state_key, state_var in a.items():
        if b.get(state_key) != None and isinstance(state_var, dict): 
            result = isInState_internal(state_var, b[state_key])
            if result == False:
                return False
            same_state_count = same_state_count + result
        elif b.get(state_key) != None and state_var != b[state_key]:
            return False
        elif b.get(state_key) != None and state_var == b[state_key]:
            same_state_count = same_state_count + 1
    return same_state_count

def isInState(a, b):
    result = isInState_internal(a, b)
    if result != False and result > 0:
        return True
    else:
        return False
    
def getNeighborActions(domain, state):
    actions = []
    for i, d in enumerate(domain):
        if isInState(d['effect'], state):
            actions.append(d)        
    return actions

def table_merge(a, b):
    #print("a(moto)", a)
    #print("b", b)
    merged = {}
    for key, var in b.items():
        #print(key, var)
        if isinstance(var, dict):
            if a.get(key) == None:
                a[key] = ""
                #print("a in dict", a)
                #a.update({key:""})
            # ここを修正!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            if isinstance(a[key], dict):
                table_merge(a[key], var)
            else:
                d = {key: var}
                a.update(d)
        else:
            #print("a[key]", a[key])
            a[key] = var
            #a.update(key:var)
            #merged[key] = var
            #print("merged", merged)
    return a

def isSameState(a, b):
    return isSameState_internal(a,b) and isSameState_internal(b, a)

def isSameState_internal(a, b):
    for key, var in b.items():
        if a.get(key) != None:
            eq = a[key] == var
            if isinstance(var, dict) and isinstance(a[key], dict):
                eq = isSameState(a[key], b)
            if eq == False:
                return False
        return True

def getIndex(lst, node):
    for l in range(len(lst)):
        if lst[l].act == node.act:
            if isSameState(lst[l].diff, node.diff):
                return l
    return 0

def compare(a, b):
    return a.score < b.score

[A] {'hand': '', 'isTop': {'b': True, 'a': False}, 'on': {'b': 'a'}}
[goal] {'on': {'b': 'a', 'a': 'table'}}
diff {'on': {'a': 'table'}}
diffcount 1
[B] {'hand': '', 'isTop': {'a': True}, 'on': {'a': 'table'}}
[goal] {'on': {'b': 'a', 'a': 'table'}}
diff {'on': {'b': 'a'}}
diffcount 1
[C] {'hand': 'b', 'on': {'b': 'hand'}, 'isTop': {'b': False}}
[goal] {'hand': 'b', 'isTop': {'a': True, 'b': False}, 'on': {'a': 'table', 'b': 'hand'}}
diff {'isTop': {'a': True}, 'on': {'a': 'table'}}
diffcount 2


In [6]:
from functools import cmp_to_key
import operator
def searchPlan(domain, state, goal):
    openlist = []
    closelist = []
    goal_node = Node(state = goal, act = None, nxt = None, cost = 0)
    goal_node.diff, goal_node.diffcount = getDiffState(goal_node.state, state)
    print("^^^^^^^^^^^^")
    print('goal_node.state', goal_node.state)
    print('state', state)
    print('goal_node.diff:', goal_node.diff)
    print('goal_node.diffcount:', goal_node.diffcount)
    goal_node.score = goal_node.diffcount
    openlist.append(goal_node)
    print("^^^^^^^^^^^^")
    
    
        
    while len(openlist) > 0:
        
        #####################################
        #for i in range(len(openlist)):
        #    print('socre', openlist[i].score)
        ####################################

        print('--------------------------------------------')
        node = openlist.pop(0)
        print("node.score", node.score)
        print("len(openlist)", len(openlist))
        
        if node.diffcount  == 0:
            print("node.state", node.state)
            return node
        
        closelist.append(node)
        
        neighbors = getNeighborActions(domain, node.state)
        for i, n in enumerate(neighbors):
            print("^^^^^^^^^^^^", i)
            print("note.state", node.state)
            print("n[effect]", n['effect'])
            diff, count = getDiffState(node.state, n['effect'])
            print("deff", diff)
            print("n[precondition]",n['precondition'])
            new_state = table_merge(diff, n['precondition'])
            print(' merged new_state:', new_state)
            print('state:', state)
            diff, diffcount = getDiffState(new_state, state)
            print('diff:', diff)
            print('diffcount:', diffcount)
            new_cost = node.cost + n['cost']
            new_score = new_cost + diffcount
            new_node = Node(state= new_state, cost= new_cost, act=n, nxt=node, score=new_score, diff=diff, diffcount=diffcount)
            print('===========================')
            print('act:', n['name'])
            print('===========================')
            #new_node = Node(state = new_state,cost = new_cost,  score = new_score, act = var, nxt = node, diff = diff, diffcount = diffcount)

            index_open = getIndex(openlist, new_node)
            if index_open > 0:
                if openlist[index_open].score > new_node.score:
                    openlist[index_open] = new_node
                    print("new_node.state", new_node.state)
            else:
                index_close = getIndex(closelist, new_node)
                if index_close == 0:
                    print("Append:new_node.state", new_node.state)
                    openlist.append(new_node)
            #print('---print openlist---')
            #for i in range(len(openlist)):
            #    print('score', openlist[i].score)
            #sorted_openlist = sorted(openlist, key=cmp_to_key(compare))
            sorted_openlist = sorted(openlist, key=operator.attrgetter('score'))
            #print('---print sorted_openlist---')
            #for i in range(len(sorted_openlist)):
            #    print('score', sorted_openlist[i].score)
            openlist = sorted_openlist
            print("^^^^^^^^^^^^", i)
    return False
            


def strips(domain, state, goal):
    node = searchPlan(domain, state, goal)
    if node == False:
        return False
    plan = []
    ############################
    ############################
    #print(node[0])
    
    while node.act != None:
        #plan.append(node[0].act.name)
        plan.append(node.act['name'])
        node = node.nxt
    return plan



state = dict(hand = "", on = dict(a = "b", b = "table", c="a"), isTop = dict(a = False, b = False, c = True))
goal = dict(on = dict(a = "table", b = "a", c = "b"))
plan1 = strips(domain, state, goal)
print("---plan---")
print(plan1)

^^^^^^^^^^^^
goal_node.state {'on': {'a': 'table', 'b': 'a', 'c': 'b'}}
state {'hand': '', 'on': {'a': 'b', 'b': 'table', 'c': 'a'}, 'isTop': {'a': False, 'b': False, 'c': True}}
goal_node.diff: {'on': {'a': 'table', 'b': 'a', 'c': 'b'}}
goal_node.diffcount: 3
^^^^^^^^^^^^
--------------------------------------------
node.score 3
len(openlist) 0
^^^^^^^^^^^^ 0
note.state {'on': {'a': 'table', 'b': 'a', 'c': 'b'}}
n[effect] {'hand': '', 'isTop': {'b': True, 'a': False}, 'on': {'b': 'a'}}
deff {'on': {'a': 'table', 'c': 'b'}}
n[precondition] {'hand': 'b', 'isTop': {'b': False, 'a': True}, 'on': {'b': 'hand'}}
 merged new_state: {'on': {'a': 'table', 'c': 'b', 'b': 'hand'}, 'hand': 'b', 'isTop': {'b': False, 'a': True}}
state: {'hand': '', 'on': {'a': 'b', 'b': 'table', 'c': 'a'}, 'isTop': {'a': False, 'b': False, 'c': True}}
diff: {'on': {'a': 'table', 'c': 'b', 'b': 'hand'}, 'hand': 'b', 'isTop': {'a': True}}
diffcount: 5
act: stackba
Append:new_node.state {'on': {'a': 'table', 'c': 'b'